In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras 
from tensorflow.keras.layers import SimpleRNN, LSTM, Dense, Dropout, Embedding,  BatchNormalization 
from tensorflow.keras.models import Sequential 
from tensorflow.keras.utils import pad_sequences

In [75]:
df = pd.read_csv("datasets/amazon.csv")

In [76]:
df.head()

,product_id,product_name,category,discounted_price,actual_price,discount_percentage,rating,rating_count,about_product,user_id,user_name,review_id,review_title,review_content,img_link,product_link
0,B07JW9H4J1,Wayona Nylon Braided USB to Lightning Fast Cha...,Computers&Accessories|Accessories&Peripherals|...,₹399,"₹1,099",64%,4.2,"24,269",High Compatibility : Compatible With iPhone 12...,"AG3D6O4STAQKAY2UVGEUV46KN35Q,AHMY5CWJMMK5BJRBB...","Manav,Adarsh gupta,Sundeep,S.Sayeed Ahmed,jasp...","R3HXWT0LRP0NMF,R2AJM3LFTLZHFO,R6AQJGUP6P86,R1K...","Satisfied,Charging is really fast,Value for mo...",Looks durable Charging is fine tooNo complains...,https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Wayona-Braided-WN3LG1-Sy...
1,B098NS6PVG,Ambrane Unbreakable 60W / 3A Fast Charging 1.5...,Computers&Accessories|Accessories&Peripherals|...,₹199,₹349,43%,4.0,"43,994","Compatible with all Type C enabled devices, be...","AECPFYFQVRUWC3KGNLJIOREFP5LQ,AGYYVPDD7YG7FYNBX...","ArdKn,Nirbhay kumar,Sagar Viswanathan,Asp,Plac...","RGIQEG07R9HS2,R1SMWZQ86XIN8U,R2J3Y1WL29GWDE,RY...","A Good Braided Cable for Your Type C Device,Go...",I ordered this cable to connect my phone to An...,https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Ambrane-Unbreakable-Char...
2,B096MSW6CT,Sounce Fast Phone Charging Cable & Data Sync U...,Computers&Accessories|Accessories&Peripherals|...,₹199,"₹1,899",90%,3.9,"7,928",【 Fast Charger& Data Sync】-With built-in safet...,"AGU3BBQ2V2DDAMOAKGFAWDDQ6QHA,AESFLDV2PT363T2AQ...","Kunal,Himanshu,viswanath,sai niharka,saqib mal...","R3J3EQQ9TZI5ZJ,R3E7WBGK7ID0KV,RWU79XKQ6I1QF,R2...","Good speed for earlier versions,Good Product,W...","Not quite durable and sturdy,https://m.media-a...",https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Sounce-iPhone-Charging-C...
3,B08HDJ86NZ,boAt Deuce USB 300 2 in 1 Type-C & Micro USB S...,Computers&Accessories|Accessories&Peripherals|...,₹329,₹699,53%,4.2,"94,363",The boAt Deuce USB 300 2 in 1 cable is compati...,"AEWAZDZZJLQUYVOVGBEUKSLXHQ5A,AG5HTSFRRE6NL3M5S...","Omkar dhale,JD,HEMALATHA,Ajwadh a.,amar singh ...","R3EEUZKKK9J36I,R3HJVYCLYOY554,REDECAZ7AMPQC,R1...","Good product,Good one,Nice,Really nice product...","Good product,long wire,Charges good,Nice,I bou...",https://m.media-amazon.com/images/I/41V5FtEWPk...,https://www.amazon.in/Deuce-300-Resistant-Tang...
4,B08CF3B7N1,Portronics Konnect L 1.2M Fast Charging 3A 8 P...,Computers&Accessories|Accessories&Peripherals|...,₹154,₹399,61%,4.2,"16,905",[CHARGE & SYNC FUNCTION]- This cable comes wit...,"AE3Q6KSUK5P75D5HFYHCRAOLODSA,AFUGIFH5ZAFXRDSZH...","rahuls6099,Swasat Borah,Ajay Wadke,Pranali,RVK...","R1BP4L2HH9TFUP,R16PVJEXKV6QZS,R2UPDB81N66T4P,R...","As good as original,Decent,Good one for second...","Bought this instead of original apple, does th...",https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Portronics-Konnect-POR-1...


In [77]:
df.drop(["product_id", "rating_count", "user_id", "user_name", "review_id", "img_link", "product_link", "product_name", "discount_percentage"], axis=1, inplace=True)

In [78]:
df['rating'] = df['rating'].replace('|', '1')
df = df.drop_duplicates()

df['discounted_price'] = df['discounted_price'].str.replace('₹', '').str.replace(',', '').astype(float)
df['actual_price'] = df['actual_price'].str.replace('₹', '').str.replace(',', '').astype(float)

df['rating'] = np.floor(df['rating'].astype(float)).astype(int)

df.head()

,category,discounted_price,actual_price,rating,about_product,review_title,review_content
0,Computers&Accessories|Accessories&Peripherals|...,399.0,1099.0,4,High Compatibility : Compatible With iPhone 12...,"Satisfied,Charging is really fast,Value for mo...",Looks durable Charging is fine tooNo complains...
1,Computers&Accessories|Accessories&Peripherals|...,199.0,349.0,4,"Compatible with all Type C enabled devices, be...","A Good Braided Cable for Your Type C Device,Go...",I ordered this cable to connect my phone to An...
2,Computers&Accessories|Accessories&Peripherals|...,199.0,1899.0,3,【 Fast Charger& Data Sync】-With built-in safet...,"Good speed for earlier versions,Good Product,W...","Not quite durable and sturdy,https://m.media-a..."
3,Computers&Accessories|Accessories&Peripherals|...,329.0,699.0,4,The boAt Deuce USB 300 2 in 1 cable is compati...,"Good product,Good one,Nice,Really nice product...","Good product,long wire,Charges good,Nice,I bou..."
4,Computers&Accessories|Accessories&Peripherals|...,154.0,399.0,4,[CHARGE & SYNC FUNCTION]- This cable comes wit...,"As good as original,Decent,Good one for second...","Bought this instead of original apple, does th..."


In [157]:
X = df.drop('rating', axis=1)
y = df['rating']

In [158]:
X['category'][0]

'Computers&Accessories|Accessories&Peripherals|Cables&Accessories|Cables|USBCables'

#  Text Cleaning

In [159]:
import nltk 
from nltk.tokenize import word_tokenize
import re

stop_words = {'a', 'about', 'above', 'after', 'again', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', 
             'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 
            'can', 'couldn', "couldn't", 'd', 'did', 'do', 'does', 'doing', 'down', 
            'during', 'each', 'few', 'for', 'from', 'further', 'had', 'has', 'have', 
         'having', 'he', 'her', 'here', 'hers', 'herself', 'him', 'himself', 'his', 'how', 'i', 
        'if', 'in', 'into', 'is', 'it', "it's", 'its', 'itself', 'just', 'll', 'm', 'ma', 'me', 'mightn', 
"mightn't", 'more', 'most', 'my', 'myself', 'needn', "needn't", 'now', 'o', 
'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 
'same', 'shan', "shan't", 'she', "she's", 'should', "should've", 'shouldn', "shouldn't", 'so', 'some', 'such', 
't', 'than', 'that', "that'll", 'the', 'their', 'theirs', 'them', 'themselves', 'then', 'there', 'these', 'they', 
'this', 'those', 'through', 'to', 'too', 'under', 'until', 'up', 've', 'very', 'was', 'wasn', "wasn't", 'we', 
'were', 'weren', "weren't", 'what', 'when', 'where', 'which', 'while', 'who', 'whom', 'why', 'will', 'with', 
'won', "won't", 'wouldn', "wouldn't", 'y', 'you', "you'd", "you'll", "you're", "you've", 'your', 'yours', 
'yourself', 'yourselves'}

In [160]:
# Review text Cleaning 
def clean_reviews(text): 
      
    # removing html brackets and other square brackets from the string using regex 
    regex = re.compile('<.*?>') # r'<.*?>' 
    text = re.sub(regex, '', text) 
  
    # removing special characters like @, #, $, etc 
    pattern = re.compile('[^a-zA-z0-9\s]') 
    text = re.sub(pattern,' ',text) 
  
    # removing numbers 
    pattern = re.compile('\d+') 
    text = re.sub(pattern,' ',text) 
  
    # converting text to lower case 
    text = text.lower() 
      
    # Tokenization of words 
    text = word_tokenize(text) 
      
    # Stop words removal 
    text = [word for word in text if not word in stop_words] 
      
    return text 

# using the clean_reviews function on the dataset 
X['category'] = X['category'].apply(clean_reviews)
X['about_product'] = X['about_product'].apply(clean_reviews)
X['review_title'] = X['review_title'].apply(clean_reviews)
X['review_content'] = X['review_content'].apply(clean_reviews)

In [161]:
X.head()

,category,discounted_price,actual_price,about_product,review_title,review_content,about_product_vector,review_title_vector,review_content_vector,review_category_vector,category_vector
0,"[computers, accessories, accessories, peripher...",399.0,1099.0,"[high, compatibility, compatible, iphone, x, x...","[satisfied, charging, really, fast, value, mon...","[looks, durable, charging, fine, toono, compla...","[-0.018069902, -0.01174388, 0.022867402, 0.277...","[-0.66339004, 0.09795828, 0.11762041, 0.755289...","[-0.3758184, 0.061008487, -0.06802536, 0.69974...","[-0.76246166, 0.4349039, 0.017926004, 1.096693...","[-0.405478, -0.42179874, -0.37622568, -0.21114..."
1,"[computers, accessories, accessories, peripher...",199.0,349.0,"[compatible, type, c, enabled, devices, androi...","[good, braided, cable, type, c, device, good, ...","[ordered, cable, connect, phone, android, auto...","[0.025377316, 0.04518587, -0.0057617347, 0.363...","[-0.6663995, 0.06344368, -0.05640848, 0.879378...","[-0.10275144, -0.07061961, -0.06186599, 0.5723...","[-0.7572267, 0.2055765, -0.06880546, 1.1375756...","[-0.405478, -0.42179874, -0.37622568, -0.21114..."
2,"[computers, accessories, accessories, peripher...",199.0,1899.0,"[fast, charger, data, sync, built, safety, pro...","[good, speed, earlier, versions, good, product...","[not, quite, durable, sturdy, https, media, am...","[0.017286016, -0.070602484, 0.049577422, 0.382...","[-0.68277055, -0.027385121, 0.104846425, 0.856...","[-0.73931867, 0.09347625, -0.0040255142, 0.614...","[-0.77920264, 0.40800148, -0.07530833, 1.14865...","[-0.405478, -0.42179874, -0.37622568, -0.21114..."
3,"[computers, accessories, accessories, peripher...",329.0,699.0,"[boat, deuce, usb, cable, compatible, smartpho...","[good, product, good, one, nice, really, nice,...","[good, product, long, wire, charges, good, nic...","[-0.18097283, 0.04908752, -0.052916944, 0.3558...","[-0.49981898, -0.13447042, -0.0791831, 0.69786...","[-0.3048437, -0.082988985, -0.011502305, 0.534...","[-0.7018286, 0.18945757, -0.16274908, 1.048123...","[-0.405478, -0.42179874, -0.37622568, -0.21114..."
4,"[computers, accessories, accessories, peripher...",154.0,399.0,"[[, charge, sync, function, ], cable, comes, c...","[good, original, decent, good, one, secondary,...","[bought, instead, original, apple, work, rs, n...","[-0.22658575, 0.0039555184, -0.09895166, 0.434...","[-0.74380493, 0.06813522, -0.016187187, 0.9648...","[-0.075287685, -0.110005386, -0.17212884, 0.53...","[-0.6976655, 0.3303009, -0.1982885, 1.274216, ...","[-0.405478, -0.42179874, -0.37622568, -0.21114..."


# Vectorization

In [163]:
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
import pandas as pd
import numpy as np

# Tokenize the text data
tokenized_data = X['about_product'] + X['review_title'] + X['review_content'] + X['category']

# Train Word2Vec model
model = Word2Vec(sentences=tokenized_data, vector_size=100, window=5, min_count=1, workers=4)

# Get Word2Vec vectors for each word in the text data
def get_avg_vector(words, model, vector_size):
    vectors = [model.wv[word] for word in words if word in model.wv]
    if vectors:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(vector_size)

# Calculate the average vector for each text entry (sentence)
df['about_product_vector'] = df['about_product'].apply(lambda x: get_avg_vector(word_tokenize(x.lower()), model, 100))
df['review_title_vector'] = df['review_title'].apply(lambda x: get_avg_vector(word_tokenize(x.lower()), model, 100))
df['review_content_vector'] = df['review_content'].apply(lambda x: get_avg_vector(word_tokenize(x.lower()), model, 100))
df['category_vector'] = df['category'].apply(lambda x: get_avg_vector(word_tokenize(x.lower()), model, 100))

# Concatenate the average vectors into a feature matrix X
X = np.concatenate([df['about_product_vector'].values.tolist(), 
                    df['review_title_vector'].values.tolist(), 
                    df['review_content_vector'].values.tolist(),
                    df['category_vector'].values.tolist(),
                    df['actual_price'].values.reshape(-1, 1),
                    df['discounted_price'].values.reshape(-1, 1)
                   ], axis=1)


    
print(X)

[[-5.18961191e-01 -1.25193819e-01  5.96784055e-01 ...  1.05741750e-02
   1.09900000e+03  3.99000000e+02]
 [-4.15003657e-01 -9.28396732e-02  6.04874671e-01 ...  1.05741750e-02
   3.49000000e+02  1.99000000e+02]
 [-4.71507072e-01  7.70574436e-02  4.01822060e-01 ...  1.05741750e-02
   1.89900000e+03  1.99000000e+02]
 ...
 [-5.48122942e-01  7.02713877e-02  3.63238573e-01 ...  6.35929823e-01
   3.08000000e+03  2.21900000e+03]
 [-5.57710171e-01 -1.34664087e-03  3.20798516e-01 ...  6.35929823e-01
   1.89000000e+03  1.39900000e+03]
 [-4.00967717e-01  1.38577774e-01  3.63403589e-01 ...  1.13629038e-02
   3.69000000e+03  2.86300000e+03]]


In [169]:
# one hot encoding 
y = pd.get_dummies(y, columns = ['rating'])

In [171]:
y 

,1,2,3,4,5
0,False,False,False,True,False
1,False,False,False,True,False
2,False,False,True,False,False
3,False,False,False,True,False
4,False,False,False,True,False
...,...,...,...,...,...
1460,False,False,False,True,False
1461,False,False,False,True,False
1462,False,False,True,False,False
1463,False,False,False,True,False


In [172]:
# Train Test split 
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42) 
  
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(1077, 402) (270, 402) (1077, 5) (270, 5)


In [174]:
# Creating a RNN model 
rnn = Sequential(name="Simple_RNN") 
rnn.add(Embedding(402, 
                        max_words, 
                        input_length=max_words)) 
  
rnn.add(SimpleRNN(128,activation='relu',return_sequences=True)) 
  
rnn.add(SimpleRNN(64,activation='relu',return_sequences=False)) 
  
rnn.add(Dense(5, activation='softmax')) 
   
# printing model summary 
print(rnn.summary())

C:\Users\kapil\anaconda3\Lib\site-packages\keras\src\layers\core\embedding.py:86: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "Simple_RNN"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ simple_rnn (SimpleRNN)               │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ simple_rnn_1 (SimpleRNN)             │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [175]:
# Compiling model 
rnn.compile( 
    loss="categorical_crossentropy", 
    optimizer='adam', 
    metrics=['accuracy'] 
) 
  
# Training the model 
history = rnn.fit(X_train, y_train, 
                        batch_size=64, 
                        epochs=2, 
                        verbose=1,  
                          validation_data = (X_test, y_test)) 
  
# Printing model score on test data 
print("Simple_RNN Score---> ", rnn.evaluate(X_test, y_test, verbose=1))

Epoch 1/2
17/17 ━━━━━━━━━━━━━━━━━━━━ 11s 333ms/step - accuracy: 0.6069 - loss: 1.1077 - val_accuracy: 0.7111 - val_loss: 0.7301
Epoch 2/2
17/17 ━━━━━━━━━━━━━━━━━━━━ 5s 287ms/step - accuracy: 0.7756 - loss: 0.5551 - val_accuracy: 0.7111 - val_loss: 0.6700
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 55ms/step - accuracy: 0.7256 - loss: 0.6431
Simple_RNN Score--->  [0.6700074672698975, 0.7111111283302307]
